In [ ]:
from selenium import webdriver
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

browser = webdriver.Edge(EdgeChromiumDriverManager().install())

# Access the website with the most popular stocks
link = 'https://br.investing.com/equities/trending-stocks'
browser.get(link)


In [ ]:
# Get the table with the most popular stocks
table = (browser.find_element_by_xpath('//*[@id="trendingInnerContent"]/table').text)
table = table.split(sep='\n')
# Remove the unnecessary spaces and the title of the table
filter_data = []
for item in table:
    item = item.split(' ')
    filtered = [x for x in item if x.strip()]
    filter_data.append(filtered)
    
del(filter_data[0])

In [ ]:
# Separate stock names from indexes
names = []
indexes = []
for item in filter_data:
    name = " ".join(item[:-7])
    names.append(name)
    index = item[-7:]
    indexes.append(index)

In [ ]:
# Putting it all together on a table
data_joint = []
for name, index in zip(names, indexes):
    dj = [name] + index
    data_joint.append(dj)
    
tb = pd.DataFrame(data_joint)
tb.columns = ['Nome', 'Último', 'Máxima', 'Mínima', 'Variação', 'Var.%', 'Vol.', 'Hora']
tb.to_excel('Atualizado.xlsx', index=False)

In [ ]:
#Send an e-mail to client with the stock table
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

user = 'programtest45@gmail.com'
receiver = 'send.email4545@gmail.com'
subject = 'Planilha Atualizada das Ações Mais Populares'

msg = MIMEMultipart()
msg['From'] = user
msg['To'] = receiver
msg['Subject'] = subject

body = """Prezado(a), <br>

Segue em anexo a planilha das ações mais populares, decida sabiamente! <br><br>

Atenciosamente,<br>

Seu ajudante"""
msg.attach(MIMEText(body, 'html', 'utf-8'))

# Attaching the file
file = './Atualizado.xlsx'
attachment = open(file, 'rb')

part = MIMEBase('application', "octet-stream")
part.set_payload(attachment.read())
encoders.encode_base64(part)
part.add_header('Content-Disposition', 'attachment; filename="Atualizado.xlsx"')
msg.attach(part)

#Sending
text = msg.as_string()
server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.login(user, 'test.program')

server.sendmail(user, receiver, text)
server.quit()